In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [3]:
system_prompt = """

You are a personal chef. The user will give you a list of ingredients they have left over in their house.

Using the web search tool, search the web for recipes that can be made with the ingredients they have.

Return recipe suggestions and eventually the recipe instructions to the user, if requested.

"""

In [4]:
from langchain.chat_models import init_chat_model
model = init_chat_model(model="meta-llama/llama-4-maverick-17b-128e-instruct",
                        model_provider="groq",
                        temperature = 0,
                        max_retries=3,
                        timeout=60,
                        max_tokens=300
                        )

In [5]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=[web_search],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)

In [6]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="I have some leftover chicken and rice. What can I make?")]},
    config # type: ignore
)

print(response['messages'][-1].content)

You can make several dishes using leftover chicken and rice. Some suggestions include:

1. Leftover Chicken and Egg Fried Rice: A quick and easy meal that's perfect for a busy weeknight.
2. Chicken Fried Rice: A simple and delicious one-pan dinner recipe that uses leftover chicken and rice.
3. Chicken and Rice Casserole: A cheesy, creamy baked casserole that's easy to make and great for using up leftover chicken and rice.
4. Rotisserie Chicken and Rice: A quick and easy skillet recipe that's perfect for a weeknight dinner.

Would you like the recipe instructions for any of these dishes?


In [7]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='I have some leftover chicken and rice. What can I make?', additional_kwargs={}, response_metadata={}, id='f21581f4-4a90-414b-b95d-3b0f720ccf05'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'qcpnbf2vn', 'function': {'arguments': '{"query":"recipes using leftover chicken and rice"}', 'name': 'web_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 776, 'total_tokens': 810, 'completion_time': 0.336679027, 'completion_tokens_details': None, 'prompt_time': 0.01687973, 'prompt_tokens_details': None, 'queue_time': 0.301187522, 'total_time': 0.353558757}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_217b6e6136', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b370e-b085-7400-adf7-71021d0b8b18-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'reci